# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 12 2023 9:43AM,264539,15,9564618,"Brookfield Pharmaceuticals, LLC",Released
1,Jun 12 2023 9:43AM,264539,15,9564619,"Brookfield Pharmaceuticals, LLC",Released
2,Jun 12 2023 9:43AM,264539,15,9564620,"Brookfield Pharmaceuticals, LLC",Released
3,Jun 12 2023 9:43AM,264539,15,9564621,"Brookfield Pharmaceuticals, LLC",Released
4,Jun 12 2023 9:43AM,264539,15,9564622,"Brookfield Pharmaceuticals, LLC",Released
5,Jun 12 2023 9:43AM,264539,15,9564623,"Brookfield Pharmaceuticals, LLC",Released
6,Jun 12 2023 9:43AM,264539,15,9565130,"Brookfield Pharmaceuticals, LLC",Released
7,Jun 12 2023 9:43AM,264539,15,9565131,"Brookfield Pharmaceuticals, LLC",Released
8,Jun 12 2023 9:43AM,264539,15,9565673,"Brookfield Pharmaceuticals, LLC",Released
9,Jun 12 2023 9:43AM,264539,15,9565674,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,264535,Released,1
16,264536,Released,3
17,264537,Released,1
18,264538,Released,1
19,264539,Released,17


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264535,NaN,1.0
264536,NaN,3.0
264537,NaN,1.0
264538,NaN,1.0
264539,NaN,17.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264516,1.0,0.0
264517,1.0,0.0
264523,8.0,11.0
264524,3.0,1.0
264525,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264516,1,0
264517,1,0
264523,8,11
264524,3,1
264525,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264516,1,0
1,264517,1,0
2,264523,8,11
3,264524,3,1
4,264525,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264516,1,
1,264517,1,
2,264523,8,11
3,264524,3,1
4,264525,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 12 2023 9:43AM,264539,15,"Brookfield Pharmaceuticals, LLC"
17,Jun 12 2023 9:42AM,264538,20,"AdvaGen Pharma, Ltd"
18,Jun 12 2023 9:33AM,264537,10,Hush Hush
19,Jun 12 2023 9:31AM,264536,10,Hush Hush
22,Jun 12 2023 9:30AM,264535,18,Seqirus
23,Jun 12 2023 9:29AM,264534,10,Hush Hush
66,Jun 12 2023 9:19AM,264533,19,ThermoFisher Scientific Inc.
67,Jun 12 2023 9:12AM,264532,10,Hush Hush
68,Jun 12 2023 9:09AM,264531,12,"SD Head USA, LLC"
69,Jun 12 2023 9:02AM,264529,18,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 12 2023 9:43AM,264539,15,"Brookfield Pharmaceuticals, LLC",,17
1,Jun 12 2023 9:42AM,264538,20,"AdvaGen Pharma, Ltd",,1
2,Jun 12 2023 9:33AM,264537,10,Hush Hush,,1
3,Jun 12 2023 9:31AM,264536,10,Hush Hush,,3
4,Jun 12 2023 9:30AM,264535,18,Seqirus,,1
5,Jun 12 2023 9:29AM,264534,10,Hush Hush,,43
6,Jun 12 2023 9:19AM,264533,19,ThermoFisher Scientific Inc.,1,
7,Jun 12 2023 9:12AM,264532,10,Hush Hush,1,
8,Jun 12 2023 9:09AM,264531,12,"SD Head USA, LLC",1,
9,Jun 12 2023 9:02AM,264529,18,"Emersa Waterbox, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 9:43AM,264539,15,"Brookfield Pharmaceuticals, LLC",17,
1,Jun 12 2023 9:42AM,264538,20,"AdvaGen Pharma, Ltd",1,
2,Jun 12 2023 9:33AM,264537,10,Hush Hush,1,
3,Jun 12 2023 9:31AM,264536,10,Hush Hush,3,
4,Jun 12 2023 9:30AM,264535,18,Seqirus,1,
5,Jun 12 2023 9:29AM,264534,10,Hush Hush,43,
6,Jun 12 2023 9:19AM,264533,19,ThermoFisher Scientific Inc.,,1
7,Jun 12 2023 9:12AM,264532,10,Hush Hush,,1
8,Jun 12 2023 9:09AM,264531,12,"SD Head USA, LLC",,1
9,Jun 12 2023 9:02AM,264529,18,"Emersa Waterbox, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 9:43AM,264539,15,"Brookfield Pharmaceuticals, LLC",17,
1,Jun 12 2023 9:42AM,264538,20,"AdvaGen Pharma, Ltd",1,
2,Jun 12 2023 9:33AM,264537,10,Hush Hush,1,
3,Jun 12 2023 9:31AM,264536,10,Hush Hush,3,
4,Jun 12 2023 9:30AM,264535,18,Seqirus,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 9:43AM,264539,15,"Brookfield Pharmaceuticals, LLC",17.0,NaN
1,Jun 12 2023 9:42AM,264538,20,"AdvaGen Pharma, Ltd",1.0,NaN
2,Jun 12 2023 9:33AM,264537,10,Hush Hush,1.0,NaN
3,Jun 12 2023 9:31AM,264536,10,Hush Hush,3.0,NaN
4,Jun 12 2023 9:30AM,264535,18,Seqirus,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 9:43AM,264539,15,"Brookfield Pharmaceuticals, LLC",17.0,0.0
1,Jun 12 2023 9:42AM,264538,20,"AdvaGen Pharma, Ltd",1.0,0.0
2,Jun 12 2023 9:33AM,264537,10,Hush Hush,1.0,0.0
3,Jun 12 2023 9:31AM,264536,10,Hush Hush,3.0,0.0
4,Jun 12 2023 9:30AM,264535,18,Seqirus,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2116244,80.0,12.0
12,264531,0.0,1.0
15,264539,17.0,0.0
18,1058116,4.0,0.0
19,529050,0.0,2.0
20,264538,1.0,0.0
22,264516,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2116244,80.0,12.0
1,12,264531,0.0,1.0
2,15,264539,17.0,0.0
3,18,1058116,4.0,0.0
4,19,529050,0.0,2.0
5,20,264538,1.0,0.0
6,22,264516,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,80.0,12.0
1,12,0.0,1.0
2,15,17.0,0.0
3,18,4.0,0.0
4,19,0.0,2.0
5,20,1.0,0.0
6,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,80.0
1,12,Released,0.0
2,15,Released,17.0
3,18,Released,4.0
4,19,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,22
Status,,,,,,,
Executing,12.0,1.0,0.0,0.0,2.0,0.0,1.0
Released,80.0,0.0,17.0,4.0,0.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,22
0,Executing,12.0,1.0,0.0,0.0,2.0,0.0,1.0
1,Released,80.0,0.0,17.0,4.0,0.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,22
0,Executing,12.0,1.0,0.0,0.0,2.0,0.0,1.0
1,Released,80.0,0.0,17.0,4.0,0.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()